In [1]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# Spam Detection  Using the Naive Bayes Algorithm

The process of creating a spam detector using the naive Bayes algorithm is split up into four steps.

  - Create a set of the most common words occurring in spam and ham (i.e. non-spam) emails.
  - For every word occurring in this set, compute the conditional probability that this words occurs in a spam or ham email.
  - Create a function that takes an email and the conditional probabilities computed before and that then computes the probability
    that the given email is spam.
  - Evaluate the <em style='color:blue;'>precision</em> and the <em style='color:blue;'>recall</em> of the spam classifier.

## Step 1: Create Word Dictionary

We need the module `os` for reading directories and the module `re` for 
<em style='color:blue;'>regular expressions</em>.

In [2]:
import os
import re
import math

An object of class <a href='https://docs.python.org/2/library/collections.html#counter-objects'>`Counter`</a> is a special form of a `dictionary` that is used for counting.  We need a counter to figure out what the most common words are.

In [3]:
from collections import Counter

The directory 
https://github.com/karlstroetmann/Artificial-Intelligence/tree/master/Python/EmailData
contains 960 emails that are divided into four subdirectories:

  - `spam-train` contains 350 spam emails for training,
  - `ham-train`  contains 350 non-spam emails for training,
  - `spam-test`  contains 130 spam emails for testing,
  - `ham-test`   contains 130 non-spam emails for testing.

Originally, this data has been collected by **Ion Androutsopoulos**.  I have found this data on the page 
http://openclassroom.stanford.edu/MainFolder/DocumentPage.php?course=MachineLearning&doc=exercises/ex6/ex6.html provided by Andrew Ng.

We declare some variables so this notebook can be adapted to other data sets.

In [4]:
spam_dir_train = 'EmailData/spam-train/'
ham__dir_train = 'EmailData/ham-train/'
spam_dir_test  = 'EmailData/spam-test/'
ham__dir_test  = 'EmailData/ham-test/'
Directories    = [spam_dir_train, ham__dir_train, spam_dir_test, ham__dir_test]

In order to compute the <em style='color:blue;'>prior probability</em> that an email is ham or spam we need to count the number of spam and ham emails.

In [5]:
no_spam    = len(os.listdir(spam_dir_train))
no_ham     = len(os.listdir(ham__dir_train))
spam_prior = no_spam / (no_spam + no_ham)
ham__prior = no_ham  / (no_spam + no_ham)
spam_prior, ham__prior

(0.5, 0.5)

I have checked that the proportion of spam and ham emails in the test directory is also $1:1$.  If the proportion of spam and ham emails in life is different from $1:1$, then we would have to use this proportion in the spam filter to be developed.

The function $\texttt{get_words}(\texttt{fn})$ takes a filename $\texttt{fn}$ as its argument.  It reads the file and returns a set of all words that are found in this file.  The words are transformed to lower case.

In [6]:
def get_words(fn):
    file = open(fn)
    text = file.read()
    text = text.lower()
    return set(re.findall(r"[\w']+", text))

Let us test this function with a small example mail.

In [7]:
get_words('EmailData/ham-train/3-380msg4.txt')

{'anyone',
 'article',
 'berkeley',
 'book',
 'consonant',
 'edu',
 'english',
 'garnet',
 'hard',
 'helpful',
 'hi',
 'interest',
 'irish',
 'laurel',
 'm',
 'modern',
 'palatal',
 'phonetics',
 'posting',
 'project',
 'recommend',
 'slender',
 'source',
 'specifically',
 'sutton',
 'thank',
 'too',
 'work'}

The function `read_all_files` reads all files contained in those directories that are stored in the list `Directories`. 
It returns a `Counter`.  For every word $w$ this counter contains the number of files that contain $w$. 

In [8]:
def read_all_files():
    Words = Counter()
    for directory in Directories:
        for file_name in os.listdir(directory):
            Words.update(get_words(directory + file_name))
    return Words

`Common_Words` is a list of the 2500 most common words found in all of our emails.

In [9]:
N            = 2500             # number of the most common words to use
Word_Counter = read_all_files()
Word_Counter

Counter({'phone': 202,
         'effort': 75,
         'power': 52,
         'ba': 13,
         'nonaccredit': 8,
         'grant': 23,
         'require': 131,
         'eminent': 9,
         'money': 187,
         'programs': 21,
         'ma': 37,
         'field': 117,
         'choice': 51,
         'advance': 81,
         'detail': 143,
         'degree': 41,
         'earn': 69,
         'necessary': 55,
         'diploma': 10,
         'prestige': 9,
         'present': 142,
         'mba': 8,
         'call': 347,
         'phd': 22,
         'through': 155,
         'university': 307,
         'knowledge': 72,
         'award': 20,
         'short': 86,
         'here': 259,
         'personal': 102,
         'experience': 123,
         'increase': 69,
         'further': 154,
         'part': 131,
         'base': 134,
         'respond': 45,
         'main': 72,
         'name': 289,
         'please': 445,
         'entitle': 13,
         'net': 100,
         'start': 173,

In [10]:
Common_Words = { w for w, _ in Word_Counter.most_common(N) }
Common_Words

{'face',
 'illegal',
 'specialize',
 'circle',
 'ibm',
 'ship',
 'recieve',
 'afford',
 'morphology',
 'review',
 'location',
 'arrange',
 'proof',
 'instruction',
 'dan',
 'brief',
 'evolution',
 'degree',
 'bind',
 'fall',
 'why',
 'reference',
 'hi',
 'genie',
 'genuine',
 'satisfy',
 'particle',
 'right',
 'did',
 'able',
 'journal',
 'speakers',
 'planet',
 'believer',
 'accommodation',
 'reception',
 'female',
 'weekly',
 'ongo',
 'mexico',
 'ideal',
 'employee',
 'secondary',
 'believe',
 'steven',
 'general',
 'profit',
 'observation',
 'each',
 'breakfast',
 'whether',
 'included',
 'michael',
 'possibility',
 'normal',
 'map',
 'secretary',
 'reservation',
 'machine',
 'download',
 'interactive',
 'instant',
 'list',
 'america',
 'rather',
 'mailer',
 'output',
 'former',
 'program',
 'problem',
 'characteristic',
 'ensure',
 'experience',
 'operator',
 'label',
 'hardware',
 'while',
 'company',
 'black',
 'anyone',
 'gb',
 'proud',
 'nijmegen',
 'candidate',
 'feel',
 'favo

## Computing the Conditional Probabilities

Having computed the most common words, we are now ready to compute the conditional probability that a given word occurs in a spam email.

The function $\texttt{get_common_words}(\texttt{fn})$ takes a filename $\texttt{fn}$ 
as its argument.  It reads the file and returns the set of all words in `Common_Words` that are found in the given file.  

In [11]:
def get_common_words(fn):
    return get_words(fn) & Common_Words

We test this function for a small email.

In [12]:
get_common_words('EmailData/ham-train/3-380msg4.txt')

{'anyone',
 'article',
 'berkeley',
 'book',
 'consonant',
 'edu',
 'english',
 'hard',
 'helpful',
 'hi',
 'interest',
 'm',
 'modern',
 'phonetics',
 'project',
 'recommend',
 'source',
 'specifically',
 'thank',
 'too',
 'work'}

The function `count_common_words` takes a string specifying a `directory`.  It returns a 
`Counter` that counts how often the words in `Common_Words` occur in any of the files in `directory`.

In [13]:
def count_commmon_words(directory):
    Words = Counter()
    for file_name in os.listdir(directory):
        Words.update(get_common_words(directory + file_name))
    return Words

Next, we compute dictionaries that store the number of occurrences in emails for every common word.

In [14]:
Spam_Counter = count_commmon_words(spam_dir_train)
Spam_Counter

Counter({'phone': 93,
         'effort': 42,
         'power': 30,
         'ba': 8,
         'grant': 12,
         'require': 64,
         'money': 140,
         'programs': 15,
         'ma': 13,
         'field': 33,
         'choice': 23,
         'advance': 20,
         'detail': 55,
         'degree': 9,
         'earn': 51,
         'necessary': 25,
         'diploma': 7,
         'present': 27,
         'call': 132,
         'phd': 6,
         'through': 75,
         'university': 15,
         'knowledge': 30,
         'award': 13,
         'short': 38,
         'here': 146,
         'personal': 67,
         'experience': 63,
         'increase': 39,
         'further': 51,
         'part': 50,
         'base': 42,
         'respond': 24,
         'main': 22,
         'name': 133,
         'please': 188,
         'entitle': 4,
         'net': 68,
         'start': 106,
         'cost': 99,
         'return': 65,
         'free': 198,
         'old': 40,
         'state': 103,
 

In [15]:
Ham__Counter = count_commmon_words(ham__dir_train)
Ham__Counter

Counter({'face': 13,
         'provide': 86,
         'europe': 20,
         'syntax': 65,
         'little': 13,
         'broad': 10,
         'morphology': 36,
         'arabic': 10,
         'hebrew': 8,
         'current': 32,
         'state': 57,
         'where': 57,
         'identify': 30,
         'description': 39,
         'centre': 29,
         'call': 119,
         'pre': 9,
         'work': 99,
         'university': 201,
         'computational': 38,
         'www': 116,
         'mike': 7,
         'iro': 7,
         'short': 25,
         'final': 33,
         'theme': 27,
         'modify': 8,
         'result': 41,
         'lab': 9,
         'workshop': 71,
         'national': 30,
         'target': 8,
         'own': 32,
         'committee': 50,
         'specifically': 8,
         'range': 29,
         'parse': 14,
         'each': 53,
         'researcher': 40,
         'examine': 16,
         'session': 47,
         'possibility': 18,
         'michael': 32,


 For every common word $w$  we compute the probability that $w$ occurs in a spam or ham email.  The formula for spam is:
 $$ P(w \in\texttt{Spam}) = \frac{\mbox{number of spam emails containing $w$}}{\mbox{number of all spam emails}} $$
 The formula for ham is similar:
 $$ P(w \in\texttt{Ham}) = \frac{\mbox{number of ham emails containing $w$}}{\mbox{number of all ham emails}} $$
 However, if we would use this formular, than a common word $w$ that, for some reason, hasn't yet occurred in any spam email, would have a 
 probability of $0$ of occurring in spam email.  Hence, our classifier would never classify an email with the word $w$ as spam.  As this
 cannot be right, we assume that there is one further spam email that contains every common word.  This 
 <em style='color:blue;'>Laplace smoothing</em> assumption changes the formula for $P(w \in\texttt{Spam})$ as follows:
 $$ P(w \in\texttt{Spam}) = \frac{\mbox{number of spam emails containing $w$ + 1}}{\mbox{number of all spam emails + 1}} $$

In [16]:
Spam_Probability = {}
Ham__Probability = {}
for w in Common_Words:
    Spam_Probability[w] = (Spam_Counter[w] + 1) / (no_spam + 1) 
    Ham__Probability[w] = (Ham__Counter[w] + 1) / (no_ham  + 1) 
Spam_Probability

{'face': 0.02564102564102564,
 'illegal': 0.03133903133903134,
 'specialize': 0.045584045584045586,
 'circle': 0.019943019943019943,
 'ibm': 0.022792022792022793,
 'ship': 0.1225071225071225,
 'recieve': 0.03418803418803419,
 'afford': 0.06837606837606838,
 'morphology': 0.002849002849002849,
 'review': 0.06837606837606838,
 'location': 0.022792022792022793,
 'arrange': 0.014245014245014245,
 'proof': 0.07407407407407407,
 'instruction': 0.18518518518518517,
 'dan': 0.005698005698005698,
 'brief': 0.02564102564102564,
 'evolution': 0.002849002849002849,
 'degree': 0.02849002849002849,
 'bind': 0.017094017094017096,
 'fall': 0.037037037037037035,
 'why': 0.18803418803418803,
 'reference': 0.037037037037037035,
 'hi': 0.07122507122507123,
 'genie': 0.02849002849002849,
 'genuine': 0.022792022792022793,
 'satisfy': 0.05698005698005698,
 'particle': 0.002849002849002849,
 'right': 0.27635327635327633,
 'did': 0.05413105413105413,
 'able': 0.1339031339031339,
 'journal': 0.03133903133903134

According to our computation, the probability that a spam email contains the word `'consonant'` is about $0.28\%$, while the probability that this word occurs in a ham email is $2.55\%$.

In [17]:
Spam_Probability['consonant'], Ham__Probability['consonant']

(0.002849002849002849, 0.02564102564102564)

For the word `'dollar'` the probability that a spam email contains this word is about $21.1\%$, while the probability that this word occurs in a ham email is $1.99\%$.

In [18]:
Spam_Probability['dollar'], Ham__Probability['dollar']

(0.21082621082621084, 0.019943019943019943)

## Deciding whether an Email is Spam

Given a file name `fn`, this function returns the probability that the message contained in the given file is spam.  

When implementing the formula 
$$\arg\max\limits_{C \in \mathcal{C}}  \left(\prod\limits_{i=1}^m P(f_i \;|\; C)\right) \cdot P(C) $$
we have to be careful, because a naive implementation will eveluate the product
$$\prod\limits_{i=1}^m P(f_i \;|\; C)$$
as the number $0$ due to numerical underflow.  The trick to compute this product is to remember that
$$ \ln(a \cdot b) = \ln(a) + \ln(b) $$
and therefore transform the product into a sum of logarithms:
$$ \prod\limits_{i=1}^m P(f_i \;|\; C) = \exp\left(\alpha + \sum\limits_{i=1}^m \ln\bigl(P(f_i \;|\; C)\bigr) \right) \cdot \exp(-\alpha)$$
Here, the constant $\alpha$ has to be chosen such that the application of the function `exp` to the value
$$ \alpha + \sum\limits_{i=1}^m \ln\bigl(P(f_i \;|\; C)\bigr) $$
does not lead to an underflow error.

As we want to compute a probability, we have to be aware that the term
$$ \left(\prod\limits_{i=1}^m P(f_i \;|\; C)\right) \cdot P(C) $$
is not the probability that the object is of class $C$ but rather is only *proportional* to this probability.  The fact that the probability
of an email being spam + the probability that the email is ham must be $1$ enables us to compute the probability.

In [19]:
def spam_probability(fn):
    log_p_spam = 0.0
    log_p_ham  = 0.0
    words = get_common_words(fn)
    for w in Common_Words:
        if w in words:
            log_p_spam += math.log(Spam_Probability[w])
            log_p_ham  += math.log(Ham__Probability[w])
        else:
            log_p_spam += math.log(1.0 - Spam_Probability[w])
            log_p_ham  += math.log(1.0 - Ham__Probability[w])
    alpha  = abs(max(log_p_spam, log_p_ham))
    p_spam = math.exp(log_p_spam + alpha) * spam_prior
    p_ham  = math.exp(log_p_ham  + alpha) * ham__prior
    return p_spam / (p_spam + p_ham)

Let us test this with a ham email.

In [20]:
spam_probability('EmailData/ham-train/3-430msg1.txt')

6.530493984338722e-29

Ok, we got this one right.  Let us check the general performance.

## Evaluate Precision and Recall

In order to evaluate the performance of this algorithm, we need to define two new concepts: <em style='color:blue;'>precision</em> and 
<em style='color:blue;'>recall</em>.  Let us call the ham emails the <em style='color:blue;'>positives</em>, while the spam emails are called the
<em style='color:blue;'>negatives</em>.  Then we define

  - <em style='color:blue;'>true positives</em>: ham emails that are classified as ham,
  - <em style='color:blue;'>false positives</em>: spam emails that are classified as ham,
  - <em style='color:blue;'>true negatives</em>: spam emails that are classified as spam,
  - <em style='color:blue;'>false negatives</em>: ham emails that are classified as spam.
  
The <em style='color:blue;'>precision</em> of the spam classifier is then defined as
$$ \texttt{precision} = \frac{\mbox{number of true positives}}{\mbox{number of true positives} + \mbox{number of false positives}} $$
Therefore, the **precision** measures the percentage of the ham emails in the set of all emails that are classified as ham.
The <em style='color:blue;'>recall</em> of the spam classifier is defined as
$$ \texttt{recall} = \frac{\mbox{number of true positives}}{\mbox{number of true positives} + \mbox{number of false negatives}} $$
Therefore, the **recall** measures the percentage of those ham emails that are indeed classified as ham.  

Usually, it is very important that the recall is high as we don't want to miss a ham email because our classifier has incorrectly classified it as a spam email.  
On the other hand, having a high precision is not that important.  After all, if $10\%$ of the emails offered to us as ham are, in fact, spam, we might tolerate this.  However, we would certainly not tolerate missing $10\%$ of our ham emails because they are incorrectly specified as spam.

The function `precission_recall` takes two directories as arguments: `spam_dir` is supposed to contain spam emails, while `ham_dir` contains ham emails.  It computes the **precision** and the **recall** of our spam classifier with respect to these test data.

In [21]:
def precission_recall(spam_dir, ham_dir):
    TN = 0 # true negatives
    FP = 0 # false positives
    for email in os.listdir(spam_dir):
        if spam_probability(spam_dir + email) > 0.5:
            TN += 1
        else:
            FP += 1
    FN = 0 # false negatives
    TP = 0 # true positives
    for email in os.listdir(ham_dir):
        if spam_probability(ham_dir + email) > 0.5:
            FN += 1
        else:
            TP += 1
    precision = TP / (TP + FP)
    recall    = TP / (TP + FN)
    accuracy  = (TN + TP) / (TN + TP + FN + FP)
    return precision, recall, accuracy

In [22]:
precission_recall(spam_dir_train, ham__dir_train)

(0.8495145631067961, 1.0, 0.9114285714285715)

In [23]:
precission_recall(spam_dir_test, ham__dir_test)

(0.7791411042944786, 0.9769230769230769, 0.85)